In [ ]:
%%bash
wget https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
tar -xzf hadoop-3.3.6.tar.gz
mv  hadoop-3.3.6/ /usr/local/

--2024-02-20 10:25:16--  https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.95.219, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 730107476 (696M) [application/x-gzip]
Saving to: ‘hadoop-3.3.6.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  245K 48m33s
    50K .......... .......... .......... .......... ..........  0% 19.0M 24m35s
   100K .......... .......... .......... .......... ..........  0%  495K 24m23s
   150K .......... .......... .......... .......... ..........  0% 21.6M 18m25s
   200K .......... .......... .......... .......... ..........  0%  508K 19m25s
   250K .......... .......... .......... .......... ..........  0% 22.3M 16m16s
   300K .......... .......... .......... .......... ..........  0% 21.8M 14m1s
   350K

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"
os.environ["PATH"] = os.environ["PATH"] + ":" + "/usr/local/hadoop-3.3.6/bin"

In [ ]:
!hdfs dfs -rm input/examen.csv

2024-02-20 11:27:31,821 INFO Configuration.deprecation: io.bytes.per.checksum is deprecated. Instead, use dfs.bytes-per-checksum
Deleted input/examen.csv


In [ ]:
%%bash
hdfs dfs -put examen.csv input

In [ ]:
!hdfs dfs -ls input

Found 1 items
-rw-r--r--   1 root root       1129 2024-02-20 10:26 input/examen.csv


In [ ]:
%%writefile UserAmountMR.java
import java.io.IOException;
import java.util.StringTokenizer;
import org.apache.hadoop.io.*;
import org.apache.hadoop.mapreduce.*;
import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.DoubleWritable;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

public class UserAmountMR
{
	public static class UserAmountMapper extends Mapper<LongWritable, Text, Text, DoubleWritable> {

		@Override
		public void map(LongWritable key, Text value, Context context) throws IOException, InterruptedException {
			String line = value.toString();
			String[] fields = line.split(",");
			if (fields.length >= 9) {
				String userId = fields[6].trim();
        double totalAmount =0;
        try
        {
		       totalAmount = Double.parseDouble(fields[2]);
        }catch(NumberFormatException ex)
        {

        }
				context.write(new Text(userId), new DoubleWritable(totalAmount));
			}
		}
	}

	public static class UserAmountReducer extends Reducer<Text, DoubleWritable, Text, DoubleWritable> {

		@Override
		public void reduce(Text key, Iterable<DoubleWritable> values, Context context) throws IOException, InterruptedException {
			double sum = 0;
			for (DoubleWritable val : values) {
				sum += val.get();
			}
			context.write(key, new DoubleWritable(sum));
		}
	}


	public static void main(String[] args) throws Exception {
		Configuration conf = new Configuration();
		Job job = Job.getInstance(conf, "user amount spent");
		job.setJarByClass(UserAmountMR.class);
		job.setMapperClass(UserAmountMapper.class);
		job.setCombinerClass(UserAmountReducer.class);
		job.setReducerClass(UserAmountReducer.class);
		job.setOutputKeyClass(Text.class);
		job.setOutputValueClass(DoubleWritable.class);
		FileInputFormat.addInputPath(job, new Path(args[0]));
		FileOutputFormat.setOutputPath(job, new Path(args[1]));
		System.exit(job.waitForCompletion(true) ? 0 : 1);
	}
}

Overwriting UserAmountMR.java


In [ ]:
%%bash
javac -cp /usr/local/hadoop-3.3.6/share/hadoop/common/*:/usr/local/hadoop-3.3.6/share/hadoop/mapreduce/* UserAmountMR.java -Xlint
jar -cvf UserAmountMR.jar *.class

added manifest
adding: contadorPrecios$AverageMapper.class(in = 1890) (out= 754)(deflated 60%)
adding: contadorPrecios$AverageReducer.class(in = 1722) (out= 733)(deflated 57%)
adding: contadorPrecios.class(in = 1584) (out= 831)(deflated 47%)
adding: UserAmountMR$UserAmountMapper.class(in = 1904) (out= 784)(deflated 58%)
adding: UserAmountMR$UserAmountReducer.class(in = 1692) (out= 702)(deflated 58%)
adding: UserAmountMR.class(in = 1543) (out= 832)(deflated 46%)


/usr/local/hadoop-3.3.6/share/hadoop/common/hadoop-common-3.3.6.jar(/org/apache/hadoop/fs/Path.class): warning: Cannot find annotation method 'value()' in type 'LimitedPrivate': class file for org.apache.hadoop.classification.InterfaceAudience not found
1 warning


In [ ]:
!hadoop jar UserAmountMR.jar UserAmountMR examen.csv ./Out25

2024-02-20 11:27:51,032 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2024-02-20 11:27:51,236 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2024-02-20 11:27:51,236 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2024-02-20 11:27:51,378 WARN mapreduce.JobResourceUploader: Hadoop command-line option parsing not performed. Implement the Tool interface and execute your application with ToolRunner to remedy this.
2024-02-20 11:27:51,541 INFO input.FileInputFormat: Total input files to process : 1
2024-02-20 11:27:51,610 INFO mapreduce.JobSubmitter: number of splits:1
2024-02-20 11:27:51,917 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local146869857_0001
2024-02-20 11:27:51,917 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-02-20 11:27:52,210 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2024-02-20 11:27:52,215 INFO mapred.LocalJobRunner: OutputCommitter set in con

In [ ]:
!hdfs dfs -cat Out25/*

178	244.73
210	401.16
321	369.02
492	434.65
603	450.21
608	728.04
948	339.47
982	252.41
990	242.69
999	423.61
user_id	0.0


In [126]:
%%bash
wget https://downloads.apache.org/pig/pig-0.17.0/pig-0.17.0.tar.gz
tar -xzf pig-0.17.0.tar.gz
cp -r pig-0.17.0/ /usr/local/

--2024-02-20 11:50:08--  https://downloads.apache.org/pig/pig-0.17.0/pig-0.17.0.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230606579 (220M) [application/x-gzip]
Saving to: ‘pig-0.17.0.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  274K 13m42s
    50K .......... .......... .......... .......... ..........  0%  550K 10m15s
   100K .......... .......... .......... .......... ..........  0% 40.0M 6m52s
   150K .......... .......... .......... .......... ..........  0% 1.23M 5m54s
   200K .......... .......... .......... .......... ..........  0%  983K 5m29s
   250K .......... .......... .......... .......... ..........  0% 73.8M 4m34s
   300K .......... .......... .......... .......... ..........  0% 79.2M 3m55s
   350K .......... .......... 

In [127]:
import os
os.environ["PIG_HOME"] = "/usr/local/pig-0.17.0"
os.environ["PATH"] = os.environ["PATH"] + ":" + "/usr/local/pig-0.17.0/bin"
os.environ["PIG_CLASSPATH"] = "/usr/local/hadoop-3.3.6/conf"

In [136]:
%%writefile calcular_gastos.pig
orders = LOAD 'input/examen.csv' USING PigStorage(',') AS (
    order_id: int,
    order_date: chararray,
    total_amount: double,
    order_item_id: int,
    quantity: double,
    unit_price: double,
    user_id: int,
    username: chararray,
    email: chararray,
    age: int,
    country: chararray
);

user_spending = FOREACH (GROUP orders BY user_id) {
    generate group AS user_id, SUM(orders.total_amount) AS total_spent;
}

STORE user_spending INTO 'user_spending_output' USING PigStorage(',');

Overwriting calcular_gastos.pig


In [137]:
!pig calcular_gastos.pig

2024-02-20 11:56:11,861 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2024-02-20 11:56:11,867 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE
2024-02-20 11:56:11,867 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType
2024-02-20 11:56:11,953 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2024-02-20 11:56:11,953 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1708430171939.log
2024-02-20 11:56:12,421 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2024-02-20 11:56:12,539 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2024-02-20 11:56:12,539 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:///
2024-02-20 11:56:12,583 [main] INFO  org.apache.pig.PigServer - Pig Script ID for the ses

In [138]:
!hdfs dfs -cat user_spending_output/*

178,244.73
210,401.16
321,369.02
492,434.65
603,450.21
608,728.04
948,339.47
982,252.41
990,242.69
999,423.61
,


In [142]:
%%writefile calcular_gastos_pais.pig
-- Load the CSV file
orders = LOAD 'input/examen.csv' USING PigStorage(',') AS (
    order_id: int,
    order_date: chararray,
    total_amount: double,
    order_item_id: int,
    quantity: double,
    unit_price: double,
    user_id: int,
    username: chararray,
    email: chararray,
    age: int,
    country: chararray
);

-- Group the data by country and user_id, and calculate the total amount spent by each user in each country
user_country_spending = FOREACH (GROUP orders BY (country, user_id)) {
    generate
        FLATTEN(group) AS (country, user_id),
        SUM(orders.total_amount) AS total_spent;
}

-- Group the data by country and find the user who spent the highest amount in each country
top_spender_per_country = FOREACH (GROUP user_country_spending BY country) {
    ordered = ORDER user_country_spending BY total_spent DESC;
    top_user = LIMIT ordered 1;
    generate group AS country, top_user.user_id AS top_spender_id, top_user.total_spent AS total_spent;
}

-- Store the result
STORE top_spender_per_country INTO 'top_spender_per_country_output' USING PigStorage(',');


Overwriting calcular_gastos_pais.pig


In [143]:
!pig calcular_gastos_pais.pig

2024-02-20 11:59:09,419 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2024-02-20 11:59:09,436 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE
2024-02-20 11:59:09,437 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType
2024-02-20 11:59:09,600 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2024-02-20 11:59:09,606 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1708430349574.log
2024-02-20 11:59:10,399 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2024-02-20 11:59:10,592 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2024-02-20 11:59:10,593 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:///
2024-02-20 11:59:10,670 [main] INFO  org.apache.pig.PigServer - Pig Script ID for the ses

In [144]:
!hdfs dfs -cat top_spender_per_country_output/*

UK,{(948)},{(339.47)}
USA,{(321)},{(369.02)}
Canada,{(608)},{(728.04)}
country,{()},{()}
Australia,{(603)},{(450.21)}
